<a href="https://colab.research.google.com/github/ankit-kothari/Data-Cleaning-Extraction/blob/master/NLP_PART1_Data_Cleaning_Extraction_and_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##GENERAL
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter
import csv
import re

##SPACY
import spacy
nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en_core_web_md')
#nlp = spacy.load('en_core_web_lg')
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.pipeline import SentenceSegmenter
from spacy import displacy


from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

##SKLEARN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


**LOADING THE DATA**

In [ ]:
data = pd.read_csv('/gdrive/My Drive/data/train.csv')
data.columns = ['rating','title','review']
data.shape

(2999999, 3)

**TEXT CLEANING**

In [ ]:
contraction_mapping = {"ain't": "is not","don't": "do not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have", "w/":"with", "cnt":"cannot", "w/o":"without","u":"you"}


def spacy_preprocessing(text):
    #print(text)
    #text = re.sub(r"\S*\w*.(com)\S*", "",text) #replaces any email or websitw with space
    text = re.sub(r"\b([a-zA-Z]{1})\b", " ", text) #replaces single random characters in the text with space
    text = re.sub(r"[^a-zA-Z]"," ",text) #replaces special characters with spaces
    text = re.sub(r"(.)\1{3,}", r"\1", text) #replaces multiple character with a word with one like pooooost will be post
    text = re.sub(r"\s{2,}", r" ", text) #replaces multiple space in the line with single space
    
    
    tokens = text.split(" ")
    #print(tokens)
    clean_tokens = [contraction_mapping[i] if i in contraction_mapping else i for i in tokens]
    text = " ".join(clean_tokens)
    #except:
    #text=text
    clean_text=[]
    for token in nlp(text):
       if (token.lemma_ != "-PRON-") & (token.text not in nlp.Defaults.stop_words):
           clean_text.append(token.text.lower())
       elif (token.lemma_ == "-PRON-")  & (token.text not in nlp.Defaults.stop_words):
           clean_text.append(token.text.lower())
       else:
           continue
    clean_string = " ".join(clean_text).lstrip()
    #print(type(clean_string))
    return clean_string

In [ ]:
data.shape

(2999999, 3)

**DATA EXTRACTION**

In [ ]:
pd.set_option('display.max_colwidth', -1)
def frametransform(rating):
  if rating == 5 or rating == 4:
    return "Good"
  elif rating == 3:
    return "Average"
  else:
    return "Bad"

data['review_category']= data['rating'].apply(frametransform)
pattern =r'\b([Aa]mazon[.+(a-z)]*)' #extracting all reviews where amazon is mentioned
data= data[data['review'].str.contains(pattern, na=False)]
data=data.dropna()
data['clean_review'] = data['review'].apply(spacy_preprocessing)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,rating,title,review,review_category,clean_review
5,1,Buyer beware,"This is a self-published book, and if you want to know why--read a few paragraphs! Those 5 star reviews must have been written by Ms. Haddon's family and friends--or perhaps, by herself! I can't imagine anyone reading the whole thing--I spent an evening with the book and a friend and we were in hysterics reading bits and pieces of it to one another. It is most definitely bad enough to be entered into some kind of a ""worst book"" contest. I can't believe Amazon even sells this kind of thing. Maybe I can offer them my 8th grade term paper on ""To Kill a Mockingbird""--a book I am quite sure Ms. Haddon never heard of. Anyway, unless you are in a mood to send a book to someone as a joke---stay far, far away from this one!",Bad,this self published book want know read paragraphs those star reviews written ms haddon family friends imagine reading thing spent evening book friend hysterics reading bits pieces it definitely bad entered kind worst book contest believe amazon sells kind thing maybe offer th grade term paper to kill mockingbird book sure ms haddon heard anyway mood send book joke stay far far away
15,1,Another Abysmal Digital Copy,"Rather than scratches and insect droppings, this one has random pixelations combined with muddy light and vague image resolution. Probably the cue should have been the packaging is straight out of your street corner bootleg dealer.If you've ever seen a reasonably good condition film copy, you know the defining visuals of his film are the crystal clear lighting contrasts in black and white. The surrounding countryside and 'old home' scenes are set with early morning ground mists or the haze of memory while the events on the bridge and in the water are bright, clear, and immediate.Here everything is dull, dark, and clouded. Or, if you remember the timbre and enunciation of Captain's commands, so are the visuals.After that, it is hard to believe this award winning, critically acclaimed film's best presentation is on YOUTUBE. Somewhere ""out there"" is a DVD that comes up to the standard of a 16mm public library reel.Just none of them appear to be on Amazon.",Bad,rather scratches insect droppings random pixelations combined muddy light vague image resolution probably cue packaging straight street corner bootleg dealer if ve seen reasonably good condition film copy know defining visuals film crystal clear lighting contrasts black white the surrounding countryside old home scenes set early morning ground mists haze memory events bridge water bright clear immediate here dull dark clouded or remember timbre enunciation captain commands visuals after hard believe award winning critically acclaimed film best presentation youtube somewhere dvd comes standard mm public library reel just appear amazon
21,5,CHECK OUT THE K6000 CHARGER!,Greetings. Kodak also has a K6000 charger which comes with 4 NiMH 2100 mAh batteries. Two batteries charge is 64 minutes and 4 batteries charge in 140 minutes. The problem is that Amazon doesn't carry them YET. Good bye.,Good,greetings kodak k charger comes nimh mah batteries two batteries charge minutes batteries charge minutes the problem amazon doesn carry yet good bye
50,2,Maybe I got a bad one.,"My experience: I got this from FM, not Amazon. The DVD player rattled pretty loudly sometimes (but not always) when I put in a DVD. I tried several. The picture quality wasn't all that great and the screen got a horizontal line thru the middle after a short time that would not go away. Plus one of the other reviewers was dead on about the remote - the layout makes no sense at all! Very poor human engineering, IMO. I brought it back to the store for a refund. I saw the same model under the Emerson name for <$400 at WM but was hesitant to buy it.",Bad,my experience got fm amazon the dvd player rattled pretty loudly dvd tried the picture quality wasn great screen got horizontal line middle short time away plus review

In [ ]:
data.shape

(112106, 5)

**SAVING THE CLEANED DATA IN A NEW CSV FILE**

In [ ]:
data['review']=data['review'].str.replace(pattern, "Amazon")
data.to_csv('/gdrive/My Drive/data/cleaned_Amazon_reviews.csv')

**LOADING THE CLEAN DATA** 

In [ ]:
pd.set_option('display.max_colwidth', -1)
pattern =r'\b([Aa]mazon[.+(a-z)]*)'
clean_data = pd.read_csv('/gdrive/My Drive/data/cleaned_Amazon_reviews.csv')
companies = (clean_data['clean_review'].str.extract(pattern))
companies.columns=['company']
companies['company'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


array(['amazon', 'amazona', 'amazonian', 'amazonmom', 'amazoni',
       'amazonians', 'amazonkindle', 'amazonthe', 'amazons', 'amazone',
       'amazontorrent', 'amazonthis', 'amazonia', 'amazonoids',
       'amazonseller', 'amazoneparrots', 'amazontimex', 'amazoncons',
       'amazongeorge', 'amazontelling', 'amazonas', 'amazonthey',
       'amazonuk', 'amazonkurt', 'amazoning', 'amazonno', 'amazong',
       'amazondotcom', 'amazonget', 'amazonheads', 'amazoners',
       'amazonic', 'amazonit', 'amazonhaving', 'amazonco',
       'amazonstoners', 'amazonshipping', 'amazonu', 'amazonmp',
       'amazonwhat', 'amazonprime', 'amazonjon', 'amazonbut',
       'amazontold', 'amazoncables', 'amazonner', 'amazonpros',
       'amazoness', 'amazontargus', 'amazonrmmcdonald', 'amazonor',
       'amazonvisually', 'amazonlet', 'amazoncame', 'amazonis',
       'amazonenjoyable', 'amazonbasics', 'amazonflyweb', 'amazonfor',
       'amazonbook', 'amazonyo', 'amazonmeguiar', 'amazonmarketplace',
      

In [ ]:
clean_data['clean_review']=clean_data['clean_review'].str.replace(pattern, "Amazon")
clean_data.head()

,Unnamed: 0,rating,title,review,review_category,clean_review
0,5,1,Buyer beware,"This is a self-published book, and if you want to know why--read a few paragraphs! Those 5 star reviews must have been written by Ms. Haddon's family and friends--or perhaps, by herself! I can't imagine anyone reading the whole thing--I spent an evening with the book and a friend and we were in hysterics reading bits and pieces of it to one another. It is most definitely bad enough to be entered into some kind of a ""worst book"" contest. I can't believe Amazon even sells this kind of thing. Maybe I can offer them my 8th grade term paper on ""To Kill a Mockingbird""--a book I am quite sure Ms. Haddon never heard of. Anyway, unless you are in a mood to send a book to someone as a joke---stay far, far away from this one!",Bad,this self published book want know read paragraphs those star reviews written ms haddon family friends imagine reading thing spent evening book friend hysterics reading bits pieces it definitely bad entered kind worst book contest believe Amazon sells kind thing maybe offer th grade term paper to kill mockingbird book sure ms haddon heard anyway mood send book joke stay far far away
1,15,1,Another Abysmal Digital Copy,"Rather than scratches and insect droppings, this one has random pixelations combined with muddy light and vague image resolution. Probably the cue should have been the packaging is straight out of your street corner bootleg dealer.If you've ever seen a reasonably good condition film copy, you know the defining visuals of his film are the crystal clear lighting contrasts in black and white. The surrounding countryside and 'old home' scenes are set with early morning ground mists or the haze of memory while the events on the bridge and in the water are bright, clear, and immediate.Here everything is dull, dark, and clouded. Or, if you remember the timbre and enunciation of Captain's commands, so are the visuals.After that, it is hard to believe this award winning, critically acclaimed film's best presentation is on YOUTUBE. Somewhere ""out there"" is a DVD that comes up to the standard of a 16mm public library reel.Just none of them appear to be on Amazon",Bad,rather scratches insect droppings random pixelations combined muddy light vague image resolution probably cue packaging straight street corner bootleg dealer if ve seen reasonably good condition film copy know defining visuals film crystal clear lighting contrasts black white the surrounding countryside old home scenes set early morning ground mists haze memory events bridge water bright clear immediate here dull dark clouded or remember timbre enunciation captain commands visuals after hard believe award winning critically acclaimed film best presentation youtube somewhere dvd comes standard mm public library reel just appear Amazon
2,21,5,CHECK OUT THE K6000 CHARGER!,Greetings. Kodak also has a K6000 charger which comes with 4 NiMH 2100 mAh batteries. Two batteries charge is 64 minutes and 4 batteries charge in 140 minutes. The problem is that Amazon doesn't carry them YET. Good bye.,Good,greetings kodak k charger comes nimh mah batteries two batteries charge minutes batteries charge minutes the problem Amazon doesn carry yet good bye
3,50,2,Maybe I got a bad one.,"My experience: I got this from FM, not Amazon The DVD player rattled pretty loudly sometimes (but not always) when I put in a DVD. I tried several. The picture quality wasn't all that great and the screen got a horizontal line thru the middle after a short time that would not go away. Plus one of the other reviewers was dead on about the remote - the layout makes no sense at all! Very poor human engineering, IMO. I brought it back to the store for a refund. I saw the same model under the Emerson name for <$400 at WM but was hesitant to buy it.",Bad,my experience got fm Amazon the dvd player rattled pretty loudly dvd tried the picture quality wasn great screen got horizontal line middle short time

**TOPIC MODELING**

In [ ]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=5, stop_words='english')
dtm = tfidf.fit_transform(clean_data['clean_review'])
nmf_model = NMF(n_components=15,random_state=42)
nmf_model.fit(dtm)
print(len(tfidf.get_feature_names()))
print(tfidf.get_feature_names()[2700])
print(nmf_model.components_.shape)

26027
breathing
(15, 26027)


In [ ]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([[tfidf.get_feature_names()[i],nmf_model.components_[index][i].round(4)] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
[['battery', 0.9314], ['got', 0.934], ['reviews', 0.9593], ['ve', 0.9597], ['return', 0.9675], ['didn', 0.9704], ['new', 0.9708], ['unit', 1.0729], ['time', 1.1914], ['buy', 1.3241], ['like', 1.3622], ['don', 1.4351], ['use', 1.4604], ['work', 1.5361], ['bought', 1.9406]]


THE TOP 15 WORDS FOR TOPIC #1
[['cover', 0.1862], ['page', 0.1935], ['information', 0.2009], ['copy', 0.2234], ['review', 0.2241], ['com', 0.2377], ['written', 0.2486], ['reviews', 0.2585], ['story', 0.2717], ['reading', 0.352], ['author', 0.3824], ['pages', 0.3916], ['books', 0.5073], ['read', 0.8374], ['book', 4.6406]]


THE TOP 15 WORDS FOR TOPIC #2
[['ray', 0.1929], ['review', 0.1944], ['copy', 0.2051], ['like', 0.213], ['video', 0.2182], ['acting', 0.219], ['love', 0.2311], ['saw', 0.2424], ['seen', 0.2943], ['watching', 0.2943], ['watched', 0.3011], ['story', 0.3061], ['film', 0.3941], ['movies', 0.6997], ['movie', 4.7848]]


THE TOP 15 WORDS FOR TOPIC #3
[['listening', 0.2336], [

**ASSIGNING TOPIC TO EACH REVIEW BASED ON THE NMF MODELING and MAPPING IT TO A TOPIC NAME**

In [ ]:
topic_category = nmf_model.transform(dtm)
clean_data['topic_category'] = topic_category.argmax(axis=1)
topic_mapping={0:'refund-and-return', 1:'books',2:'movies',3:'music',4:'video-quality',5:'shipping',6:'product-quality',7:'cost',8:'refund-and-return',9:'games', 10:'books', 11:'tv-series',12:'packaging',13:'deals',14:'product-quality'}
clean_data['topic_name']=clean_data['topic_category'].map(topic_mapping) 
clean_data= clean_data[['review','clean_review','review_category','topic_category', 'topic_name']]

**GROUPPING THE DATAFRAME BY TOPIC NAME AND REVIEW CATEGORY (GOOD, BAD, AVERAGE) AND SORTING IT BY THE WORST PERCENTAGE OF BAD REVIEWS**

In [ ]:
clean_data_grouped = clean_data.groupby(['topic_name','review_category']).agg(
  {

    'review_category':'count'

  })

clean_data_grouped= clean_data_grouped.unstack().reset_index()
clean_data_grouped['percentage_bad']= (clean_data_grouped['review_category']['Bad']/(clean_data_grouped['review_category']['Average']+clean_data_grouped['review_category']['Good']+clean_data_grouped['review_category']['Bad'])*100)
clean_data_grouped['percentage_bad']=clean_data_grouped['percentage_bad'].round(2)
clean_data_grouped.sort_values(by=['percentage_bad'], ascending=False)
clean_data_grouped.index.rename('ID', inplace=True)
clean_data_grouped

topic_name review_category              percentage_bad
review_category                            Average    Bad  Good               
ID                                                                            
0                books              2618            8714   4941  53.55        
1                cost               1358            2654   5114  29.08        
2                deals              931             2139   4910  26.80        
3                games              262             823    624   48.16        
4                movies             852             2286   1717  47.09        
5                music              1228            3367   3173  43.34        
6                packaging          865             2664   1214  56.17        
7                product-quality    3152            11178  5565  56.18        
8                refund-and-return  3163            15050  4807  65.38        
9                shipping           1085            5912   1614  68.66        
10               tv-series          425             1374   1124  47.01        
11               video-quality      764             3166   1273  60.85

**NEXT UP IN NLP PART 2: WE WILL LOOK IN INDEPTH OF TOPIC CATEGORY *PRODUCT-QUALITY* for AMAZON REVIEWS AND MODEL THEM FOR GOOD BAD OR AVERAGE '**

In [ ]:
amazon_product_quality = clean_data[clean_data['topic_name'].isin(['movies','music','books','games','video-quality','refund-and-return'])]
amazon_product_quality.to_csv('/gdrive/My Drive/data/amazon_products.csv')